In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys

Append the `grt_reader` repo so we can read modules from sibling directories.


In [4]:
sys.path.append(os.path.abspath(os.path.join("..")))

In [5]:
import numpy as np
import pandas as pd
from utils.constants import question_ids_map
from utils.questions import (
    construct_single_question_for_ai,
)

from notebooks.utils.process_lease import process_lease

/home/david/grt/grt_reader/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
bucket_name = os.getenv("BUCKET_NAME", "")
index_name = os.getenv("INDEX_NAME", "")

In [7]:
print(f"Bucket name: {bucket_name}")
print(f"Index name: {index_name}")

Bucket name: gaaprt-reader-dev
Index name: development-gaaprt-reader-openai


Retrieve questions


In [8]:
questions_df = pd.read_excel("../data/asc_842/grt_ai_reader_questions_v1.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [9]:
answers_df = pd.read_excel("../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [10]:
question_id = question_ids_map["end_date"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)

## Calculating the accuracy for all the leases


In [11]:
numbers_list = [str(i).zfill(3) for i in range(1, 101)]

## Experiment 1: gpt-3.5-turbo, text-embedding-ada-002, unstructured outputs


In [12]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")
        print(f"Answer: {answer}")
        print(f"Real answer: {formatted_answer}")
        if answer == formatted_answer:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf


KeyboardInterrupt: 

## Experiment 2: gpt-4o-mini, text-embedding-3-small, structured outputs


In [14]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            model="gpt-4o-mini",
            embedding_model="text-embedding-3-small",
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")
        print(f"Answer: {answer.answer_date}")
        print(f"Real answer: {formatted_answer}")
        if answer.answer_date == formatted_answer:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: 2028-05-01
Real answer: 2027-01-25
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: 2026-03-31
Real answer: 2029-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: 2021-07-31
Real answer: 2021-07-31
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.pdf
Stored eafit/lease005.pdf in vector database with namespace eafit_lease005
Answer: 2012-03-31
Real answer: 2005-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease006.pdf
Stored eafit/lease006.pdf in vector database with namespace eafit_lease006
Answer: 2023-

0.4772727272727273

## Experiment 3: gpt-4o-2024-08-06, text-embedding-3-small, structured outputs


In [12]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            model="gpt-4o-2024-08-06",
            embedding_model="text-embedding-3-small",
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")
        print(f"Answer: {answer.answer_date}")
        print(f"Real answer: {formatted_answer}")
        if answer.answer_date == formatted_answer:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: None
Real answer: 2027-01-25
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: 2029-03-31
Real answer: 2029-03-31
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: 2021-07-31
Real answer: 2021-07-31
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.pdf
Stored eafit/lease005.pdf in vector database with namespace eafit_lease005
Answer: 2012-03-31
Real answer: 2005-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease006.pdf
Stored eafit/lease006.pdf in vector database with namespace eafit_lease006
Answer: 2023-10-06
Re

0.4659090909090909

## Experiment 4: gpt-3.5-turbo, text-embedding-3-small, JSON mode


In [12]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            force_structured_output=True,
            model="gpt-3.5-turbo",
            embedding_model="text-embedding-3-small",
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")
        print(f"Answer: {answer}")
        print(f"Real answer: {formatted_answer}")
        if answer == formatted_answer:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: Unknown
Real answer: 2027-01-25
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: 2024-04-01
Real answer: 2029-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: N/A
Real answer: 2021-07-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.pdf
Stored eafit/lease005.pdf in vector database with namespace eafit_lease005
Answer: 2022-12-31
Real answer: 2005-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease006.pdf
Stored eafit/lease006.pdf in vector database with namespace eafit_lease006
Answer: N/A
Real answ

0.14772727272727273

## Experiment 5: gpt-3.5-turbo, text-embedding-3-large


In [11]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            model="gpt-3.5-turbo",
            embedding_model="text-embedding-3-large",
            index_name=os.getenv("INDEX_LATEST_NAME", ""),
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")
        print(f"Answer: {answer}")
        print(f"Real answer: {formatted_answer}")
        if answer == formatted_answer:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: 2027-05-01
Real answer: 2027-01-25
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: 2024-04-01
Real answer: 2029-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: 2021-07-31
Real answer: 2021-07-31
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.pdf
Stored eafit/lease005.pdf in vector database with namespace eafit_lease005
Error decoding JSON: Expecting ',' delimiter: line 1 column 10 (char 9)
No answer found
Answer: None
Real answer: 2005-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease006.pdf
Stored

0.375

## Experiment 6: gpt-4o-mini, text-embedding-3-large, structured outputs


In [12]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            model="gpt-4o-mini",
            embedding_model="text-embedding-3-large",
            index_name=os.getenv("INDEX_LATEST_NAME", ""),
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")
        print(f"Answer: {answer.answer_date}")
        print(f"Real answer: {formatted_answer}")
        if answer.answer_date == formatted_answer:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: YYYY-MM-DD
Real answer: 2027-01-25
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: 2024-03-01
Real answer: 2029-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: 2021-07-31
Real answer: 2021-07-31
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.pdf
Stored eafit/lease005.pdf in vector database with namespace eafit_lease005
Answer: 2012-03-31
Real answer: 2005-03-31
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease006.pdf
Stored eafit/lease006.pdf in vector database with namespace eafit_lease006
Answer: 2025-

0.5113636363636364